<p align=center><img src=https://d31uz8lwfmyn8g.cloudfront.net/Assets/logo-henry-white-lg.png><p>

# <h1 align=center> **PROYECTO INDIVIDUAL Nº1** </h1>

# <h1 align=center>**`Machine Learning Operations (MLOps)`**</h1>

<p align="center">
<img src="https://user-images.githubusercontent.com/67664604/217914153-1eb00e25-ac08-4dfa-aaf8-53c09038f082.png"  height=300>

In [12]:
#### DataGuru ####
import pandas as pd
import numpy as np
import ast

class DataGuru:
    """
    Clase para procesar y manipular datos.
    """

    def __init__(self, path):
        """
        Inicializa una instancia de DataGuru.

        Args:
            path (str): Ruta del archivo CSV.
        """
        self.path = path
        self.data = None

    def read_csv(self):
        """
        Lee el archivo CSV y carga los datos en un DataFrame.
        """
        self.data = pd.read_csv(self.path, dtype=str, encoding='UTF-8', decimal='.', quotechar='"')

    def normalize_data(self, dict_columns, dict_list_columns):
        """
        Normaliza las columnas que contienen diccionarios y listas de diccionarios.

        Args:
            dict_columns (list): Lista de nombres de columnas con estructura de diccionarios.
            dict_list_columns (list): Lista de nombres de columnas con estructura de listas de diccionarios.
        """
        for column_name in dict_columns:
            self.normalize_dictionary_column(column_name)

        for column_name in dict_list_columns:
            self.normalize_dictionary_list_column(column_name)

    def normalize_dictionary_column(self, column_name):
        """
        Normaliza una columna que contiene un diccionario.

        Args:
            column_name (str): Nombre de la columna a normalizar.
        """
        self.data[column_name] = self.data[column_name].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
        keys = set().union(*self.data[column_name].apply(lambda x: x.keys() if isinstance(x, dict) else []))
        new_columns = [f"{column_name}_{key}" for key in keys]
        new_columns_data = []

        for _, row in self.data.iterrows():
            if isinstance(row[column_name], dict):
                values = [row[column_name].get(key) for key in keys]
            else:
                values = [None] * len(keys)
            new_columns_data.append(dict(zip(new_columns, values)))

        self.data = pd.concat([self.data.drop(column_name, axis=1), pd.DataFrame(new_columns_data)], axis=1)

    def normalize_dictionary_list_column(self, column_name):
        """
        Normaliza una columna que contiene una lista de diccionarios.

        Args:
            column_name (str): Nombre de la columna a normalizar.
        """
        self.data[column_name] = self.data[column_name].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        keys = set().union(*self.data[column_name].apply(lambda x: {key for item in x if isinstance(item, dict) for key in item.keys()} if isinstance(x, list) else []))
        new_columns = [f"{column_name}_{key}" for key in keys]
        new_columns_data = []

        for _, row in self.data.iterrows():
            if isinstance(row[column_name], list):
                row_data = {new_col: None for new_col in new_columns}
                for item in row[column_name]:
                    if isinstance(item, dict):
                        for key, value in item.items():
                            new_col_name = f"{column_name}_{key}"
                            row_data[new_col_name] = value
                new_columns_data.append(row_data)
            else:
                new_columns_data.append({new_col: None for new_col in new_columns})

        self.data = pd.concat([self.data.drop(column_name, axis=1), pd.DataFrame(new_columns_data)], axis=1)

    def trim_spaces(self):
        """
        Elimina los espacios en blanco al inicio y final de los valores en todas las celdas del DataFrame.
        """
        self.data = self.data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    def assign_dtypes_num(self, column_list):
        """
        Asigna los tipos de datos numéricos a las columnas especificadas.

        Args:
            column_list (list): Lista de nombres de columnas a convertir en tipos numéricos.
        """
        for column_name in column_list:
            self.data[column_name] = pd.to_numeric(self.data[column_name], errors='coerce', downcast='integer')

    def assign_dtypes_date(self, column_list):
        """
        Asigna el tipo de dato fecha a las columnas especificadas.

        Args:
            column_list (list): Lista de nombres de columnas a convertir en tipos de fecha.
        """
        for column_name in column_list:
            self.data[column_name] = pd.to_datetime(self.data[column_name], format='%Y-%m-%d', errors='coerce')

    def replace_null_values(self):
        """
        Reemplaza los valores nulos en las columnas de tipo objeto con una cadena vacía ('') y los valores nulos
        en las columnas de tipo float con 0.
        """
        object_columns = self.data.select_dtypes(include='object').columns
        float_columns = self.data.select_dtypes(include=['float64', 'float32']).columns
        self.data[object_columns] = self.data[object_columns].fillna('')
        self.data[float_columns] = self.data[float_columns].fillna(0)

    def drop_na(self, column_list):
        """
        Elimina las filas que contienen valores nulos en las columnas especificadas.

        Args:
            column_list (list): Lista de nombres de columnas para verificar los valores nulos y eliminar las filas correspondientes.
        """
        for column in column_list:
            self.data.dropna(subset=[column], inplace=True)

    def redondear_decimales(self, column_list, d):
        """
        Redondea los valores decimales en las columnas especificadas.

        Args:
            column_list (list): Lista de nombres de columnas para redondear los valores decimales.
            d (int): Número de decimales a redondear.
        """
        for column in column_list:
            self.data[column] = self.data[column].round(d)

    def add_release_year_column(self):
        """
        Agrega una columna 'release_year' al DataFrame, que contiene el año extraído de la columna 'release_date'.
        """
        self.data['release_year'] = self.data['release_date'].dt.year
        self.data['release_year'] = self.data['release_year'].astype(str)

    def drop_columns(self, column_names):
        """
        Elimina las columnas especificadas del DataFrame.

        Args:
            column_names (list): Lista de nombres de columnas a eliminar.
        """
        for column_name in column_names:
            self.data = self.data.drop(column_name, axis=1)

    def check_null_and_inf_values(self, columns):
        """
        Verifica si las columnas especificadas contienen valores nulos o infinitos y muestra un mensaje correspondiente.

        Args:
            columns (list): Lista de nombres de columnas para verificar los valores nulos e infinitos.
        """
        for column in columns:
            null_values = self.data[column].isnull()
            inf_values = np.isinf(self.data[column])
            
            tiene_nulos = "tiene" if null_values.any() else "no tiene"
            tiene_infinitos = "tiene" if inf_values.any() else "no tiene"
            
            print(f"La columna '{column}' {tiene_nulos} valores nulos y {tiene_infinitos} valores infinitos.")

    def create_return_column(self, a, b, c):
        """
        Crea una nueva columna en el DataFrame que contiene el resultado de la división de dos columnas existentes.

        Args:
            a (str): Nombre de la primera columna.
            b (str): Nombre de la segunda columna.
            c (str): Nombre de la columna de resultado.
        """
        self.data[c] = np.divide(self.data[a], self.data[b], out=np.zeros_like(self.data[a]), where=self.data[b] != 0).round(2)

    def print_data(self):
        """
        Imprime las primeras filas del DataFrame.
        """
        print(self.data.head())
    
    def info_data(self):
        """
        Muestra información sobre el DataFrame, incluyendo el tipo de datos de cada columna y el recuento de valores no nulos.
        """
        self.data.info()

    def columns_data(self):
        """
        Devuelve una lista de los nombres de las columnas en el DataFrame.

        Returns:
            list: Lista de nombres de columnas.
        """
        return self.data.columns
    
    def get_data(self):
        """
        Devuelve el DataFrame actual.

        Returns:
            pandas.DataFrame: DataFrame actual.
        """
        return self.data


In [13]:
# movies_dataset.csv: Instanciar clase y ejecutar métodos
## Parámetros
path_in_movies = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML/Data Set/movies_dataset.csv'

# Instanciar clase DataGuru
DataGuru = DataGuru(path_in_movies)

# Leer archivo CSV y crear DataFrame
DataGuru.read_csv()

# Lista de columnas a normalizar (Dtype : dict, Dtype: list of dict)
(dict_columns, dict_list_columns) = (['belongs_to_collection'],['genres', 'production_companies', 'production_countries', 'spoken_languages'])
# Normalizar columnas con dict o list de dict
DataGuru.normalize_data(dict_columns, dict_list_columns)

# Limpiar espacios antes y después de caracteres
DataGuru.trim_spaces()

# Lista de columnas a convertir a int
num_columns = ['id', 'runtime', 'vote_count', 'budget', 'popularity', 'revenue', 'vote_average']
# Asignar DTypes a columnas int, float y date
DataGuru.assign_dtypes_num(num_columns)

# Lista de columnas a convertir a YYYY-MM-DD
date_columns = ['release_date']
# Asignar DTypes a columnas int, float y date
DataGuru.assign_dtypes_date(date_columns)

# Eliminar columnas innecesarias
drop_columns = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'original_language', 'runtime', 'status',
                'tagline', 'belongs_to_collection_backdrop_path', 'belongs_to_collection_name', 'belongs_to_collection_id', 'belongs_to_collection_poster_path',
                'genres_id', 'production_companies_id', 'production_countries_iso_3166_1', 'spoken_languages_name', 'spoken_languages_iso_639_1']
DataGuru.drop_columns(drop_columns)

# Reemplazar valores nulos a str='', num='0'
DataGuru.replace_null_values()

# Eliminar filas con valores nulos en 'release_date'
drop_na = ['release_date']
DataGuru.drop_na(drop_na)

# Redondear columnas float a 4 decimales
round_float = ['popularity']
DataGuru.redondear_decimales(round_float, 4)

# Redondear columnas int a 0 decimales
round_int = ['id', 'budget', 'revenue', 'vote_count']
DataGuru.redondear_decimales(round_int, 0)

# Agregar columna 'release_year'
DataGuru.add_release_year_column()

# Crear columna 'return' usando a/b = c
(a, b, c) = ('revenue', 'budget', 'return')
DataGuru.create_return_column(a, b, c)

# Revisar nulos e infinitos en columnas específicas
check_null_inf = ['id', 'budget', 'revenue', 'vote_count', 'return']
DataGuru.check_null_and_inf_values(check_null_inf)

# Guardar DataFrame normalizado
df_movies = DataGuru.get_data()
df_movies_norm = df_movies  # Copia normalizada

# # Exportar DataFrame a CSV
# path_out_movies = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML_API_1.0/Data Set'
# DataGuru.save_csv(path_out_movies)


La columna 'id' no tiene valores nulos y no tiene valores infinitos.
La columna 'budget' no tiene valores nulos y no tiene valores infinitos.
La columna 'revenue' no tiene valores nulos y no tiene valores infinitos.
La columna 'vote_count' no tiene valores nulos y no tiene valores infinitos.
La columna 'return' no tiene valores nulos y no tiene valores infinitos.


In [14]:
df_movies_norm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   budget                     45376 non-null  float64       
 1   id                         45376 non-null  float64       
 2   overview                   45376 non-null  object        
 3   popularity                 45376 non-null  float64       
 4   release_date               45376 non-null  datetime64[ns]
 5   revenue                    45376 non-null  float64       
 6   title                      45376 non-null  object        
 7   vote_average               45376 non-null  float64       
 8   vote_count                 45376 non-null  float64       
 9   genres_name                45376 non-null  object        
 10  production_companies_name  45376 non-null  object        
 11  production_countries_name  45376 non-null  object        
 12  rele

In [15]:
#### DataGuru2 ####
import pandas as pd
import ast

class DataGuru2:
    def __init__(self, path):
        self.path = path
        self.data = self.read_csv()
        self.cast_data = None
        self.crew_data = None

    def read_csv(self):
        """
        Lee el archivo CSV y devuelve un DataFrame.
        """
        self.data = pd.read_csv(self.path, encoding='UTF-8')
        return self.data

    def rename_id_column(self):
        """
        Renombra la columna 'id' a 'credits_id' en el DataFrame.
        """
        self.data.rename(columns={'id': 'credits_id'}, inplace=True)

    def desanidar_cast_and_agregar_job(self):
        """
        Desanida la columna 'cast' del DataFrame, crea un nuevo DataFrame 'cast_data' y agrega la columna 'job' cuando 'character' no es nulo.
        """
        self.data['cast'] = self.data['cast'].apply(ast.literal_eval)
        self.cast_data = pd.json_normalize(self.data.to_dict(orient='records'), 'cast', ['credits_id'])

        # Eliminar columnas innecesarias de cast_data
        drop_columns = ['cast_id', 'credit_id', 'gender', 'id', 'order', 'profile_path']
        self.cast_data = self.cast_data.drop(drop_columns, axis=1)

        # Agregar columna 'job' a cast_data cuando 'character' no es nulo
        self.cast_data['job'] = self.cast_data['character'].apply(lambda x: 'Actor' if pd.notnull(x) else None)

    def desanidar_crew(self):
        """
        Desanida la columna 'crew' del DataFrame y crea un nuevo DataFrame 'crew_data'.
        """
        self.data['crew'] = self.data['crew'].apply(ast.literal_eval)
        self.crew_data = pd.json_normalize(self.data.to_dict(orient='records'), 'crew', ['credits_id'])

        # Eliminar columnas innecesarias de crew_data
        drop_columns = ['credit_id', 'department', 'gender', 'profile_path']
        self.crew_data = self.crew_data.drop(drop_columns, axis=1)

        return self.crew_data


    def crear_dataframe_credits_norm(self):
        """
        Crea un DataFrame 'df_credits_norm' unificando las columnas 'job', 'name' y 'credits_id' de cast_data y crew_data.
        """
        # Obtener valores únicos de cast_data
        cast_unique = self.cast_data[['credits_id', 'job', 'name']].drop_duplicates()

        # Obtener valores únicos de crew_data
        crew_unique = self.crew_data[['credits_id', 'job', 'name']].drop_duplicates()

        # Concatenar los DataFrames de valores únicos
        df_credits_norm = pd.concat([cast_unique, crew_unique], ignore_index=True)

        return df_credits_norm
    
    def get_data(self,data):
        if data == 'cast':
            return self.cast_data
        if data == 'crew':
            return self.crew_data
        if data == 'credits':
            return self.data


In [16]:
# credits.csv: Instanciar clase y ejecutar métodos

# Ruta del archivo credits.csv
path_in_credits = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML/Data Set/credits.csv'

# Instanciar la clase DataGuru2 y pasar la ruta del archivo como argumento
DataGuru2 = DataGuru2(path_in_credits)

# Renombrar la columna 'id' a 'credits_id' en el DataFrame
DataGuru2.rename_id_column()

# Desanidar la columna 'cast' del DataFrame y agregar la columna 'job' cuando 'character' no es nulo
DataGuru2.desanidar_cast_and_agregar_job()

# Desanidar la columna 'crew' del DataFrame
DataGuru2.desanidar_crew()

# Crear el DataFrame 'df_credits_norm' unificando las columnas 'job', 'name' y 'credits_id'
df_credits_norm = DataGuru2.crear_dataframe_credits_norm()



In [17]:
df_credits_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024400 entries, 0 to 1024399
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   credits_id  1024400 non-null  object
 1   job         1024400 non-null  object
 2   name        1024400 non-null  object
dtypes: object(3)
memory usage: 23.4+ MB


In [7]:
# Exportar DataFrames a CSV

path_out_movies = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML_API_1.1/Movies_API/Data Set/df_movies_norm.csv'
path_out_credits = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML_API_1.1/Movies_API/Data Set/df_credits_norm.csv'

df_movies_norm.to_csv(path_out_movies, index=False, encoding='UTF-8')

df_credits_norm.to_csv(path_out_credits, index=False, encoding='UTF-8')


In [348]:
# Exportar CVS a DataFrames, entorno local

path_in_movies = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML_API_1.1/Data Set/df_movies_norm.csv'
path_in_credits = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/d_moviesML_API_1.1/Data Set/df_credits_norm.csv'

df_movies_norm = pd.read_csv(path_in_movies, encoding='UTF-8', decimal='.')
df_credits_norm = pd.read_csv(path_in_credits,encoding='UTF-8')
# Convertir la columna 'release_date' a tipo datetime
df_movies_norm['release_date'] = pd.to_datetime(df_movies_norm['release_date'])
def replace_null_values(data):
    """
    Reemplaza los valores nulos en las columnas de tipo objeto con una cadena vacía ('') y los valores nulos
    en las columnas de tipo float con 0.
    """
    object_columns = data.select_dtypes(include='object').columns
    float_columns = data.select_dtypes(include=['float64', 'float32']).columns
    data[object_columns] = data[object_columns].fillna('')
    data[float_columns] = data[float_columns].fillna(0)
replace_null_values(df_movies_norm)



In [9]:
# Exportar CVS a DataFrames, entorno web
import requests
url_movies = 'https://github.com/fran-as/Movies_API/blob/369937996ef8e6f385297e2dc9530ddd0f549379/Data%20Set/df_movies_norm.csv'
url_credits = 'https://github.com/fran-as/Movies_API/blob/369937996ef8e6f385297e2dc9530ddd0f549379/Data%20Set/df_credits_norm.csv'
movies = requests.get(url_movies).content
credits = requests.get(url_movies).content
df_movies_norm = pd.read_csv(path_in_movies, encoding='UTF-8', decimal='.')
df_credits_norm = pd.read_csv(path_in_credits,encoding='UTF-8')
# Convertir la columna 'release_date' a tipo datetime
df_movies_norm['release_date'] = pd.to_datetime(df_movies_norm['release_date'], errors='coerce')

def replace_null_values(data):
    """
    Reemplaza los valores nulos en las columnas de tipo objeto con una cadena vacía ('') y los valores nulos
    en las columnas de tipo float con 0.
    """
    object_columns = data.select_dtypes(include='object').columns
    float_columns = data.select_dtypes(include=['float64', 'float32']).columns
    data[object_columns] = data[object_columns].fillna('')
    data[float_columns] = data[float_columns].fillna(0)
replace_null_values(df_movies_norm)

/var/folders/jn/3qyrgyf52x36fs6fmyy8mjzr0000gn/T/ipykernel_17309/2468693949.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_norm = pd.read_csv(path_in_movies, encoding='UTF-8', decimal='.')


In [88]:
#Análisis de la consistencia de datos
resultado = df_credits_norm[(df_credits_norm['name'].str.contains('Tom Hanks')) & (df_credits_norm['job'] == 'Actor')]
resultado = resultado.merge(df_movies, left_on='credits_id', right_on='id')
resultado.head(10)



,credits_id,job,name,budget,id,overview,popularity,release_date,revenue,title,vote_average,vote_count,genres_name,production_companies_name,production_countries_name,release_year,return
0,862,Actor,Tom Hanks,30000000.0,862.0,"Led by Woody, Andy's toys live happily in his ...",21.9469,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Family,Pixar Animation Studios,United States of America,1995,12.45
1,568,Actor,Tom Hanks,52000000.0,568.0,The true story of technical troubles that scut...,18.7615,1995-06-30,355237933.0,Apollo 13,7.3,1637.0,Drama,Universal Pictures,United States of America,1995,6.83
2,13,Actor,Tom Hanks,55000000.0,13.0,A man with a low IQ has accomplished great thi...,48.3072,1994-07-06,677945399.0,Forrest Gump,8.2,8147.0,Romance,Paramount Pictures,United States of America,1994,12.33
3,9800,Actor,Tom Hanks,26000000.0,9800.0,No one would take his case until one man was w...,13.3730,1993-12-22,206678440.0,Philadelphia,7.6,1019.0,Drama,Clinica Estetico,United States of America,1993,7.95
4,858,Actor,Tom Hanks,21000000.0,858.0,A young boy who tries to set his dad up on a d...,10.2349,1993-06-24,227799884.0,Sleepless in Seattle,6.5,630.0,Romance,TriStar Pictures,United States of America,1993,10.85
5,32562,Actor,Tom Hanks,0.0,32562.0,This documentary highlights the historical con...,6.7847,1996-01-30,0.0,The Celluloid Closet,7.0,24.0,Documentary,Telling Pictures,United States of America,1996,0.00
6,9591,Actor,Tom Hanks,0.0,9591.0,A Pennsylvania band scores a hit in 1964 and r...,8.6858,1996-10-04,34585416.0,That Thing You Do!,6.8,230.0,Drama,Clavius Base,United States of America,1996,0.00
7,857,Actor,Tom Hanks,70000000.0,857.0,"As U.S. troops storm the beaches of Normandy, ...",21.7581,1998-07-24,481840909.0,Saving Private Ryan,7.9,5148.0,War,Mark Gordon Productions,United States of America,1998,6.88
8,11974,Actor,Tom Hanks,18000000.0,11974.0,When secretive new neighbors move in next door...,7.8892,1989-02-17,36602000.0,The 'Burbs,6.6,312.0,Thriller,Imagine Entertainment,United States of America,1989,2.03
9,2619,Actor,Tom Hanks,8000000.0,2619.0,A successful businessman falls in love with th...,8.4416,1984-03-09,69821334.0,Splash,6.1,328.0,Romance,Touchstone Pictures,United States of America,1984,8.73


In [349]:
# Clase MovieAnalizer para consultas sobre los dataframes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

class MovieAnalyzer:

    def cantidad_filmaciones_mes(self, mes: str):
        # Convertir el nombre del mes a minúsculas
        mes = mes.lower()

        # Asignar el número correspondiente al mes
        meses_dict = {
            'enero': '01',
            'febrero': '02',
            'marzo': '03',
            'abril': '04',
            'mayo': '05',
            'junio': '06',
            'julio': '07',
            'agosto': '08',
            'septiembre': '09',
            'octubre': '10',
            'noviembre': '11',
            'diciembre': '12'
        }
        mes_numero = meses_dict.get(mes)

        # Verificar si el mes es válido
        if mes_numero:
            # Convertir la columna 'release_date' a tipo string
            df_movies_norm['release_date'] = df_movies_norm['release_date'].dt.strftime('%Y-%m-%d')

            # Filtrar las filas del DataFrame que corresponden al mes consultado
            filtrado = df_movies_norm[df_movies_norm['release_date'].str[5:7] == mes_numero]

            # Obtener la cantidad de películas estrenadas en el mes
            cantidad = len(filtrado)

            # Retornar el resultado
            return {'mes': mes, 'cantidad': cantidad}
        else:
            return {'error': f'El mes "{mes}" no es válido.'}

    def cantidad_filmaciones_dia(self, dia: str):
        # Convertir el nombre del día a minúsculas
        dia = dia.lower()

        # Asignar el número correspondiente al día de la semana
        dias_dict = {
            'lunes': 0,
            'martes': 1,
            'miércoles': 2,
            'jueves': 3,
            'viernes': 4,
            'sábado': 5,
            'domingo': 6
        }
        dia_numero = dias_dict.get(dia)

        # Verificar si el día es válido
        if dia_numero is not None:
            # Convertir la columna 'release_date' a tipo datetime
            df_movies_norm['release_date'] = pd.to_datetime(df_movies_norm['release_date'])

            # Filtrar las filas del DataFrame que corresponden al día consultado
            filtrado = df_movies_norm[df_movies_norm['release_date'].dt.dayofweek == dia_numero]

            # Obtener la cantidad de películas estrenadas en el día
            cantidad = len(filtrado)

            # Retornar el resultado
            return {'día': dia, 'cantidad': cantidad}
        else:
            return f'El día "{dia}" no es válido.'

    def score_titulo(self, titulo: str):
        # Convertir el título a minúsculas
        titulo = titulo.lower()

        # Filtrar las películas por título en el DataFrame df_movies_norm (comparación insensible a mayúsculas y minúsculas)
        filtrado = df_movies_norm[df_movies_norm['title'].str.lower() == titulo]

        # Verificar si se encontraron películas
        if len(filtrado) > 0:
            resultados = []
            for index, row in filtrado.iterrows():
                # Obtener el año de estreno y el score/popularidad de cada película
                anio = row['release_year']
                popularidad = row['popularity']

                # Agregar el resultado a la lista de resultados
                resultados.append({'titulo': row['title'], 'anio': anio, 'popularidad': popularidad})

            # Retornar la lista de resultados
                return resultados
        else:
            return f'No se encontró ninguna película con título "{titulo}" en el dataset.'

    def votos_titulo(self, titulo: str):
        # Convertir el título a minúsculas
        titulo = titulo.lower()

        # Filtrar las películas por título en el DataFrame df_movies_norm (comparación insensible a mayúsculas y minúsculas)
        filtrado = df_movies_norm[df_movies_norm['title'].str.lower() == titulo]

        # Verificar si se encontró la película
        if len(filtrado) > 0:
            votos_total = filtrado.iloc[0]['vote_count']
            voto_promedio = filtrado.iloc[0]['vote_average']

            # Verificar si tiene al menos 2000 valoraciones
            if votos_total >= 2000:
                return {'titulo': filtrado.iloc[0]['title'], 'voto_total': votos_total, 'voto_promedio': voto_promedio}
            else:
                return f'La película "{filtrado.iloc[0]["title"]}" no cumple con la condición de tener al menos 2000 valoraciones.'
        else:
            return f'No se encontró ninguna película con título "{titulo}" en el dataset.'
            
    def get_actor(self, nombre_actor: str):
        # Convertir el nombre del actor a minúsculas
        nombre_actor = nombre_actor.lower()

        # Filtrar el DataFrame df_credits_norm por el nombre del actor y el trabajo 'Actor' (comparación insensible a mayúsculas y minúsculas)
        filtrado = df_credits_norm[(df_credits_norm['name'].str.lower() == nombre_actor) & (df_credits_norm['job'] == 'Actor')]

        # Obtener la cantidad de filmaciones del actor
        cantidad_filmaciones = len(filtrado)

        # Verificar si el actor ha participado en al menos una filmación
        if cantidad_filmaciones > 0:
            # Obtener los créditos de las filmaciones del actor
            creditos_actor = filtrado['credits_id'].unique()

            # Filtrar el DataFrame df_movies_norm por los créditos del actor
            peliculas_actor = df_movies_norm[df_movies_norm['id'].isin(creditos_actor)]

            # Calcular el retorno total y el promedio de retorno excluyendo las filmaciones con retorno igual a cero
            retorno_total = peliculas_actor['return'].sum().round(2)
            cantidad_filmaciones_retorno = len(peliculas_actor[peliculas_actor['return'] != 0])
            retorno_promedio = (retorno_total / cantidad_filmaciones_retorno).round(2)

            return {'actor': nombre_actor, 'cantidad_filmaciones': cantidad_filmaciones, 'retorno_total': retorno_total, 'retorno_promedio': retorno_promedio}
        else:
            return f'No se encontró ninguna filmación para el actor "{nombre_actor}" en el dataset.'

    def get_director(self, nombre_director: str):
        # Convertir el nombre del director a minúsculas
        nombre_director = nombre_director.lower()

        # Filtrar el DataFrame df_credits_norm por el nombre del director y el trabajo 'Director' (comparación insensible a mayúsculas y minúsculas)
        filtrado = df_credits_norm[(df_credits_norm['name'].str.lower() == nombre_director) & (df_credits_norm['job'] == 'Director')]

        # Obtener la cantidad de filmaciones del director
        cantidad_filmaciones = len(filtrado)

        # Verificar si el director ha dirigido al menos una película
        if cantidad_filmaciones > 0:
            # Obtener los créditos de las filmaciones del director
            creditos_director = filtrado['credits_id'].unique()

            # Filtrar el DataFrame df_movies_norm por los créditos del director
            peliculas_director = df_movies_norm[df_movies_norm['id'].isin(creditos_director)]

            # Calcular el éxito total del director
            retorno_total_director = peliculas_director['return'].sum().round(2)

            # Crear una lista para almacenar la información de cada película
            peliculas_info = []

            # Recorrer cada película del director
            for index, pelicula in peliculas_director.iterrows():
                info_pelicula = {
                    'titulo': pelicula['title'],
                    'fecha_lanzamiento': pelicula['release_date'].strftime('%Y-%m-%d'),
                    'retorno_individual': float(pelicula['return']),
                    'costo': pelicula['budget'],
                    'ganancia': pelicula['revenue']
                }
                peliculas_info.append(info_pelicula)

            return {
                'director': nombre_director,
                'retorno_total_director': retorno_total_director,
                'peliculas': peliculas_info
            }
        else:
            return f'No se encontró ninguna película dirigida por "{nombre_director}" en el dataset.'

    def recomendar_peliculas(self, pelicula_entrada: str):
        # Convertir el título de la película de entrada a minúsculas
        pelicula_entrada = pelicula_entrada.lower()

        # Convertir los títulos en el DataFrame a minúsculas
        df_movies_norm['title_lower'] = df_movies_norm['title'].str.lower()

        # Crear el vectorizador TF-IDF para 'overview'
        vectorizer = TfidfVectorizer()
        vector_matrix = vectorizer.fit_transform(df_movies_norm['overview'].astype(str))

        # Obtener la matriz de características
        features = vectorizer.get_feature_names_out()
        matriz_caracteristicas = pd.DataFrame(vector_matrix.toarray(), columns=features)

        # Obtener el índice de la película de entrada
        indice_pelicula_entrada = df_movies_norm[df_movies_norm['title_lower'] == pelicula_entrada].index[0]

        # Obtener las características de la película de entrada
        pelicula_entrada_caracteristicas = matriz_caracteristicas.iloc[indice_pelicula_entrada]

        # Calcular la similitud del coseno entre las características de la película de entrada y las demás películas
        similitudes = cosine_similarity(matriz_caracteristicas, pelicula_entrada_caracteristicas.values.reshape(1, -1))

        # Escalar la similitud a un rango de 0 a 1
        scaler = MinMaxScaler()
        similitudes_escaladas = scaler.fit_transform(similitudes)

        # Asignar la similitud como puntaje a cada película
        df_movies_norm['puntaje'] = similitudes_escaladas.flatten()

        # Ordenar las películas por puntaje en orden descendente
        peliculas_recomendadas = df_movies_norm.sort_values(by='puntaje', ascending=False).head(5)

        # Crear el JSON con la información de las películas recomendadas
        peliculas_json = peliculas_recomendadas[['title', 'release_year', 'vote_average', 'popularity', 'genres_name']].to_json(orient='records')

        return peliculas_json


In [350]:

MovieAnalyzer = MovieAnalyzer()
MovieAnalyzer.cantidad_filmaciones_mes('enero')


{'mes': 'enero', 'cantidad': 5912}

In [351]:
MovieAnalyzer.cantidad_filmaciones_dia('lunes')

{'día': 'lunes', 'cantidad': 3503}

In [352]:
MovieAnalyzer.score_titulo('titanic')

[{'titulo': 'Titanic', 'anio': 1997, 'popularidad': 26.8891}]

In [353]:
MovieAnalyzer.votos_titulo('titanic')

{'titulo': 'Titanic', 'voto_total': 7770.0, 'voto_promedio': 7.5}

In [354]:
MovieAnalyzer.get_actor('tom HANKS')

{'actor': 'tom hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 178.85,
 'retorno_promedio': 4.36}

In [355]:
MovieAnalyzer.get_director('James CAMERON')

{'director': 'james cameron',
 'retorno_total_director': 54.24,
 'peliculas': [{'titulo': 'True Lies',
   'fecha_lanzamiento': '1994-07-14',
   'retorno_individual': 3.29,
   'costo': 115000000.0,
   'ganancia': 378882411.0},
  {'titulo': 'Terminator 2: Judgment Day',
   'fecha_lanzamiento': '1991-07-01',
   'retorno_individual': 5.2,
   'costo': 100000000.0,
   'ganancia': 520000000.0},
  {'titulo': 'The Abyss',
   'fecha_lanzamiento': '1989-08-09',
   'retorno_individual': 1.29,
   'costo': 70000000.0,
   'ganancia': 90000098.0},
  {'titulo': 'Aliens',
   'fecha_lanzamiento': '1986-07-18',
   'retorno_individual': 9.91,
   'costo': 18500000.0,
   'ganancia': 183316455.0},
  {'titulo': 'The Terminator',
   'fecha_lanzamiento': '1984-10-26',
   'retorno_individual': 12.25,
   'costo': 6400000.0,
   'ganancia': 78371200.0},
  {'titulo': 'Titanic',
   'fecha_lanzamiento': '1997-11-18',
   'retorno_individual': 9.23,
   'costo': 200000000.0,
   'ganancia': 1845034188.0},
  {'titulo': 'Pir

In [356]:
MovieAnalyzer.recomendar_peliculas('avatar')

: 

: 

In [ ]:
from fastapi import FastAPI
import pandas as pd

class MovieAnalyzer:
    def cantidad_filmaciones_mes(self, mes: str):
        """
        Devuelve la cantidad de filmaciones realizadas en un mes específico.

        Args:
            mes (str): El nombre del mes.

        Returns:
            dict: Un diccionario con el nombre del mes y la cantidad de filmaciones.

        """
        # Convertir el nombre del mes a minúsculas
        mes = mes.lower()

        # Asignar el número correspondiente al mes
        meses_dict = {
            'enero': '01',
            'febrero': '02',
            'marzo': '03',
            'abril': '04',
            'mayo': '05',
            'junio': '06',
            'julio': '07',
            'agosto': '08',
            'septiembre': '09',
            'octubre': '10',
            'noviembre': '11',
            'diciembre': '12'
        }
        mes_numero = meses_dict.get(mes)

        # Verificar si el mes es válido
        if mes_numero:
            # Filtrar las filas del DataFrame que corresponden al mes consultado
            filtrado = df_movies_norm[df_movies_norm['release_date'].dt.strftime('%m') == mes_numero]

            # Obtener la cantidad de películas estrenadas en el mes
            cantidad = len(filtrado)

            # Retornar el resultado
            return {'mes': mes, 'cantidad': cantidad}
        else:
            return {'error': f'El mes "{mes}" no es válido.'}

    def cantidad_filmaciones_dia(self, dia: str):
        """
        Devuelve la cantidad de filmaciones realizadas en un día de la semana específico.

        Args:
            dia (str): El nombre del día.

        Returns:
            dict: Un diccionario con el nombre del día y la cantidad de filmaciones.

        """
        # Convertir el nombre del día a minúsculas
        dia = dia.lower()

        # Asignar el número correspondiente al día de la semana
        dias_dict = {
            'lunes': 0,
            'martes': 1,
            'miercoles': 2,
            'jueves': 3,
            'viernes': 4,
            'sabado': 5,
            'domingo': 6
        }
        dia_numero = dias_dict.get(dia)

        # Verificar si el día es válido
        if dia_numero is not None:
            # Filtrar las filas del DataFrame que corresponden al día consultado
            filtrado = df_movies_norm[df_movies_norm['release_date'].dt.dayofweek == dia_numero]

            # Obtener la cantidad de películas estrenadas en el día
            cantidad = len(filtrado)

            # Retornar el resultado
            return {'dia': dia, 'cantidad': cantidad}
        else:
            return f'El día "{dia}" no es válido.'

    def score_titulo(self, titulo: str):
        """
        Devuelve la información de las películas con un título específico.

        Args:
            titulo (str): El título de la película.

        Returns:
            list: Una lista con la información de las películas encontradas.

        """
        # Filtrar las películas por título en el DataFrame df_movies_norm
        filtrado = df_movies_norm[df_movies_norm['title'] == titulo]

        # Verificar si se encontraron películas
        if len(filtrado) > 0:
            resultados = []
            for index, row in filtrado.iterrows():
                # Obtener el año de estreno y el score/popularidad de cada película
                anio = row['release_year']
                popularidad = row['popularity']

                # Agregar el resultado a la lista de resultados
                resultados.append({'titulo': titulo, 'anio': anio, 'popularidad': popularidad})

            # Retornar la lista de resultados
            return resultados
        else:
            return f'No se encontró ninguna película con título "{titulo}" en el dataset.'

    def votos_titulo(self, titulo: str):
        """
        Devuelve la información de votos de una película específica.

        Args:
            titulo (str): El título de la película.

        Returns:
            dict: Un diccionario con la información de votos de la película.

        """
        # Filtrar las películas por título en el DataFrame df_movies_norm
        filtrado = df_movies_norm[df_movies_norm['title'] == titulo]

        # Verificar si se encontró la película
        if len(filtrado) > 0:
            votos_total = filtrado.iloc[0]['vote_count']
            voto_promedio = filtrado.iloc[0]['vote_average']

            # Verificar si tiene al menos 2000 valoraciones
            if votos_total >= 2000:
                return {'titulo': titulo, 'voto_total': votos_total, 'voto_promedio': voto_promedio}
            else:
                return f'La película "{titulo}" no cumple con la condición de tener al menos 2000 valoraciones.'
        else:
            return f'No se encontró ninguna película con título "{titulo}" en el dataset.'

    def get_actor(self, nombre_actor: str):
        """
        Devuelve la información de un actor específico.

        Args:
            nombre_actor (str): El nombre del actor.

        Returns:
            dict: Un diccionario con la información del actor.

        """
        # Filtrar el DataFrame df_credits_norm por el nombre del actor y el trabajo 'Actor'
        filtrado = df_credits_norm[(df_credits_norm['name'] == nombre_actor) & (df_credits_norm['job'] == 'Actor')]

        # Obtener la cantidad de filmaciones del actor
        cantidad_filmaciones = len(filtrado)

        # Verificar si el actor ha participado en al menos una filmación
        if cantidad_filmaciones > 0:
            # Obtener los créditos de las filmaciones del actor
            creditos_actor = filtrado['credits_id'].unique()

            # Filtrar el DataFrame df_movies_norm por los créditos del actor
            peliculas_actor = df_movies_norm[df_movies_norm['id'].isin(creditos_actor)]

            # Calcular el retorno total y el promedio de retorno excluyendo las filmaciones con retorno igual a cero
            retorno_total = peliculas_actor['return'].sum().round(2)
            cantidad_filmaciones_retorno = len(peliculas_actor[peliculas_actor['return'] != 0])
            retorno_promedio = (retorno_total / cantidad_filmaciones_retorno).round(2)

            return {'actor': nombre_actor, 'cantidad_filmaciones': cantidad_filmaciones, 'retorno_total': retorno_total, 'retorno_promedio': retorno_promedio}
        else:
            return f'No se encontró ninguna filmación para el actor "{nombre_actor}" en el dataset.'

    def get_director(self, nombre_director: str):
        """
        Devuelve la información de un director específico.

        Args:
            nombre_director (str): El nombre del director.

        Returns:
            dict: Un diccionario con la información del director.

        """
        # Filtrar el DataFrame df_credits_norm por el nombre del director y el trabajo 'Director'
        filtrado = df_credits_norm[(df_credits_norm['name'] == nombre_director) & (df_credits_norm['job'] == 'Director')]

        # Obtener la cantidad de filmaciones del director
        cantidad_filmaciones = len(filtrado)

        # Verificar si el director ha dirigido al menos una película
        if cantidad_filmaciones > 0:
            # Obtener los créditos de las filmaciones del director
            creditos_director = filtrado['credits_id'].unique()

            # Filtrar el DataFrame df_movies_norm por los créditos del director
            peliculas_director = df_movies_norm[df_movies_norm['id'].isin(creditos_director)]

            # Calcular el éxito total del director
            retorno_total_director = peliculas_director['return'].sum().round(2)

            # Crear una lista para almacenar la información de cada película
            peliculas_info = []

            # Recorrer cada película del director
            for index, pelicula in peliculas_director.iterrows():
                info_pelicula = {
                    'titulo': pelicula['title'],
                    'fecha_lanzamiento': pelicula['release_date'].strftime('%Y-%m-%d'),
                    'retorno_individual': float(pelicula['return']),
                    'costo': pelicula['budget'],
                    'ganancia': pelicula['revenue']
                }
                peliculas_info.append(info_pelicula)

            return {
                'director': nombre_director,
                'retorno_total_director': retorno_total_director,
                'peliculas': peliculas_info
            }
        else:
            return f'No se encontró ninguna película dirigida por "{nombre_director}" en el dataset.'

    def recomendar_peliculas(self, pelicula_entrada: str, df_movies_norm: pd.DataFrame):
        """
        Recomienda películas similares a una película de entrada.

        Args:
            pelicula_entrada (str): El título de la película de entrada.
            df_movies_norm (pd.DataFrame): El DataFrame con la información de las películas.

        Returns:
            dict: Un diccionario con las recomendaciones de películas.

        """
        # Filtrar el DataFrame df_movies_norm por la película de entrada
        filtrado = df_movies_norm[df_movies_norm['title'] == pelicula_entrada]

        # Verificar si se encontró la película de entrada
        if len(filtrado) > 0:
            pelicula_entrada = filtrado.iloc[0]

            # Obtener los géneros de la película de entrada
            generos_entrada = pelicula_entrada['genres'].split('|')

            # Filtrar el DataFrame df_movies_norm por los géneros de la película de entrada
            filtrado_generos = df_movies_norm[df_movies_norm['genres'].apply(lambda x: any(genero in x for genero in generos_entrada))]

            # Excluir la película de entrada del DataFrame filtrado_generos
            filtrado_generos = filtrado_generos[filtrado_generos['title'] != pelicula_entrada['title']]

            # Ordenar las películas por el score y la popularidad en orden descendente
            recomendaciones = filtrado_generos.sort_values(by=['score', 'popularity'], ascending=False)

            # Seleccionar las 5 primeras películas como recomendaciones
            recomendaciones = recomendaciones.head(5)

            # Crear una lista para almacenar la información de cada película recomendada
            peliculas_recomendadas = []

            # Recorrer cada película recomendada
            for index, pelicula in recomendaciones.iterrows():
                info_pelicula = {
                    'titulo': pelicula['title'],
                    'fecha_lanzamiento': pelicula['release_date'].strftime('%Y-%m-%d'),
                    'score': pelicula['score'],
                    'popularidad': pelicula['popularity']
                }
                peliculas_recomendadas.append(info_pelicula)

            return {
                'pelicula_entrada': pelicula_entrada['title'],
                'recomendaciones': peliculas_recomendadas
            }
        else:
            return f'No se encontró ninguna película con título "{pelicula_entrada}" en el dataset.'


# Cargar los datos en DataFrames
df_movies = pd.read_csv('movies.csv')
df_credits = pd.read_csv('credits.csv')

# Instanciar la clase MovieAnalyzer
movie_analyzer = MovieAnalyzer()

# Crear una instancia de FastAPI
app = FastAPI()

# Definir las rutas de la API
@app.get("/")
def read_root():
    return {"API": "Análisis de películas"}

@app.get("/filmaciones/mes/{mes}")
def cantidad_filmaciones_mes(mes: str):
    return movie_analyzer.cantidad_filmaciones_mes(mes)

@app.get("/filmaciones/dia/{dia}")
def cantidad_filmaciones_dia(dia: str):
    return movie_analyzer.cantidad_filmaciones_dia(dia)

@app.get("/peliculas/score/{titulo}")
def score_titulo(titulo: str):
    return movie_analyzer.score_titulo(titulo)

@app.get("/peliculas/votos/{titulo}")
def votos_titulo(titulo: str):
    return movie_analyzer.votos_titulo(titulo)

@app.get("/actores/{nombre_actor}")
def get_actor(nombre_actor: str):
    return movie_analyzer.get_actor(nombre_actor)

@app.get("/directores/{nombre_director}")
def get_director(nombre_director: str):
    return movie_analyzer.get_director(nombre_director)

@app.get("/recomendar/{pelicula_entrada}")
def recomendar_peliculas(pelicula_entrada: str):
    return movie_analyzer.recomendar_peliculas(pelicula_entrada, df_movies_norm)
